In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
train_data = pd.read_csv('data/train_cleaned.csv')

train_data.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,1.0,0.0,530001,38.1,132.0,24.0,1.0,4.0,2.0,2.0,...,57.0,8.5,3.0,3.4,1.0,2209,0,0,0.0,0.0
1,1.0,0.0,533836,37.5,88.0,12.0,1.0,3.0,5.0,2.0,...,33.0,64.0,3.0,2.0,1.0,2208,0,0,0.0,1.0
2,1.0,0.0,529812,38.3,120.0,28.0,1.0,4.0,6.0,1.0,...,37.0,6.4,3.0,3.4,1.0,5124,0,0,0.0,2.0
3,1.0,0.0,5262541,37.1,72.0,30.0,0.0,4.0,6.0,2.0,...,53.0,7.0,1.0,3.9,1.0,2208,0,0,1.0,2.0
4,0.0,0.0,5299629,38.0,52.0,48.0,3.0,3.0,4.0,1.0,...,47.0,7.3,1.0,2.6,0.0,0,0,0,1.0,2.0


opdel data i features og labels

In [3]:
X = train_data.drop(columns=['outcome'])
y = train_data['outcome']

X_train, X_test,\
    y_train, y_test = train_test_split(X, y,
                                       test_size=0.2,
                                       random_state=42)

In [4]:
def create_model(optimizer='adam', neurons=64, epochs=50, batch_size=32, validation_split=0.2):
    model = keras.Sequential([
        keras.layers.Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split, verbose=0)
    return model

In [5]:
model = KerasClassifier(build_fn=create_model, verbose=2)

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Data scaler
    ('clf', model)  # Logistisk regression model
])

param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'neurons': [32, 64, 128],
    'epochs': [50, 100],
    'batch_size': [16, 32],
    'validation_split': [0.2, 0.3]
}



C:\Users\chris\AppData\Local\Temp\ipykernel_17948\466602679.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [6]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')

grid_result = grid.fit(X_train, y_train)

11/11 [==============================] - 0s 502us/step


In [7]:
# Get the best model and evaluate on the test data
best_model = grid_result.best_estimator_

y_pred = best_model.predict(X_test)

# Evaluate multiple metrics on the test data
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test), average='macro', multi_class='ovr')

# Print the best hyperparameters and test metrics
print("Best Hyperparameters:", grid_result.best_params_)
print("Test Metrics:")
print("  Accuracy:", accuracy)
print("  Precision:", precision)
print("  Recall:", recall)
print("  F1 Score:", f1)
print("  ROC AUC Score:", roc_auc)

8/8 [==============================] - 0s 717us/step
Best Hyperparameters: {'batch_size': 16, 'epochs': 50, 'neurons': 128, 'optimizer': 'adam', 'validation_split': 0.2}
Test Metrics:
  Accuracy: 0.4939271255060729
  Precision: 0.49728997289972904
  Recall: 0.3395061728395062
  F1 Score: 0.2319213937742548
  ROC AUC Score: 0.5412229646803364


c:\Users\chris\miniconda3\envs\ML-ENV2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
